In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [ ]:
# Dataset是抽象类，无法实例化
# 需要由子类继承
# 主要实现 索引 数据长度 功能
class Diabetes(Dataset):
    def __init__(self, filepath):
        xy = np.loadtxt(filepath, delimiter=',', dtype=np.float32)
        self.len = xy.shape[0]
        self.x_data = torch.from_numpy(xy[:,:-1])
        self.y_data = torch.from_numpy(xy[:,[-1]])

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]
    
    def __len__(self):
        return self.len


In [ ]:
if __name__ == '__main__':
    # 实例化
    dataset = Diabetes(r"../dataset/diabetes.csv.gz") 
    train_loader = DataLoader(dataset=dataset,
                            batch_size=32,
                            shuffle=True,
                            num_workers=0)


In [ ]:
# 构建模型
class Model(torch.nn.Module):
    def __init__(self):
        super(Model,self).__init__()
        self.Linear1 = torch.nn.Linear(8,6)
        self.Linear2 = torch.nn.Linear(6,4)
        self.Linear3 = torch.nn.Linear(4,1)
        self.sigmoid = torch.nn.Sigmoid()

    def forward(self,x):
        x = self.sigmoid(self.Linear1(x))
        x = self.sigmoid(self.Linear2(x))
        x = self.sigmoid(self.Linear3(x))
        return x

In [ ]:

model = Model()
criterion = torch.nn.BCELoss(reduction='mean')
optimiser = torch.optim.Adam(model.parameters(),lr=0.001)

In [ ]:
n = 100
list = []
for epoch in range(n):
    for i, data in enumerate(train_loader, 0):
        x, y = data

        y_pred = model(x)
        loss = criterion(y_pred, y)

        # 保存日志
        print(f"第 {epoch+1} 轮: {loss.detach().numpy():.4f}")
        
        optimiser.zero_grad()
        loss.backward()
        optimiser.step()
    list.append(loss.detach().numpy())
  
plt.plot(list)